#Retrieving the Dataset

In [58]:
# Supressing the warning messages
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Mounting your drive to colab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# unzipping the dataset

import zipfile as z
zip_ref = z.ZipFile("/content/drive/My Drive/the-movies-dataset.zip", "r")
zip_ref.extractall("/content/")
zip_ref.close()

In [48]:
# read the CSV file
import pandas as pd

md = pd.read_csv('/content/movies_metadata.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
md.shape

(45466, 24)

In [33]:
md.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


md dataset Contains information on 45,000 movies featured in the Full MovieLens dataset.

In [34]:
print(md.columns)
len(md.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


24

In the kaggle data escription it is mentioned that the below three rows contain most of the nan values,I decide to remove the rows from dataset

In [49]:
md.iloc[[19730, 29503, 35587]]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,False,6.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,False,7.0,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,False,4.3,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
# droping rows by index
md = md.drop([19730, 29503, 35587])

links_small data contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset and I am working on the small data set.

In [51]:
links_small=pd.read_csv("/content/links_small.csv")
links_small

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9120,162672,3859980,402672.0
9121,163056,4262980,315011.0
9122,163949,2531318,391698.0
9123,164977,27660,137608.0


In [52]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [53]:
links_small

0          862
1         8844
2        15602
3        31357
4        11862
         ...  
9120    402672
9121    315011
9122    391698
9123    137608
9124    410803
Name: tmdbId, Length: 9112, dtype: int64

In [55]:
#performing look up opertion on all movies that are present in links_small dataset
md['id'] = md['id'].astype('int')
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 24)

So, smd is the dataset I will be working on.

#Description based recommendation

Desciption implies the tagline and the overview column, so my tendency is to bring them together and apply a vectorization menthod on the Description keywords.

In [56]:
smd['tagline']

0                                                      NaN
1                Roll the dice and unleash the excitement!
2        Still Yelling. Still Fighting. Still Ready for...
3        Friends are the people who let you be yourself...
4        Just When His World Is Back To Normal... He's ...
                               ...                        
40224                      A god incarnate. A city doomed.
40503              The band you know. The story you don't.
44821                        Pokémon: Spell of the Unknown
44826                                                  NaN
45265                                                  NaN
Name: tagline, Length: 9099, dtype: object

In [59]:
smd['tagline'] = smd['tagline'].fillna(' ')

In [60]:
smd['tagline']

0                                                         
1                Roll the dice and unleash the excitement!
2        Still Yelling. Still Fighting. Still Ready for...
3        Friends are the people who let you be yourself...
4        Just When His World Is Back To Normal... He's ...
                               ...                        
40224                      A god incarnate. A city doomed.
40503              The band you know. The story you don't.
44821                        Pokémon: Spell of the Unknown
44826                                                     
45265                                                     
Name: tagline, Length: 9099, dtype: object

In [61]:
# Merging Overview and title together


smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna(' ')

In [62]:
smd.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'description'],
      dtype='object')

The Description Column is being created.

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

Applying tfidf vectorizer the feature vectorization is done.

In [67]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim[0]

array([1.        , 0.01667298, 0.        , ..., 0.        , 0.0067194 ,
       0.        ])

finding the movie similarity based on description vector keys.The method works on the principle of cosine similarity.

In [64]:
smd = smd.reset_index()
titles = smd['title']
# finding indices of every title
indices = pd.Series(smd.index, index=titles)

In [65]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)#sorts in descending order
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [68]:
#testing result.
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
1582    The Godfather: Part III
3509                       Made
4196         Johnny Dangerously
8387                 The Family
7760              Henry's Crime
7591                    Machete
9076            The Maid's Room
5593                     Eulogy
2412             American Movie
Name: title, dtype: object

#Metadata based recommendation

All the associated metadata is given along with tha main dataset, at first i import/read those files.

In [103]:
credits = pd.read_csv('/content/credits.csv')
keywords = pd.read_csv('/content/keywords.csv')

In [104]:
credits.shape

(45476, 3)

In [105]:
credits.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [106]:
keywords.shape

(46419, 2)

In [107]:
keywords.head(3)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."


so the two files are stringified jason file.I have to extract metadata from the given format.

And also the shape of credits dataset and the shape of keywords dataset is different. I have to have the common dataset belongs to the all three inter-related dataset.

In [108]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [109]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [110]:
smd2 = md[md['id'].isin(links_small)]
smd2.shape

(11367, 33)

So, I extract the small dataset having all the metadata informations.

In [111]:
smd2.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast_x', 'crew_x', 'keywords_x',
       'cast_y', 'crew_y', 'keywords_y', 'cast', 'crew', 'keywords'],
      dtype='object')

In [112]:
import ast

smd2['cast'] = smd2['cast'].apply(ast.literal_eval)#string to list
smd2['crew'] = smd2['crew'].apply(ast.literal_eval)
smd2['keywords'] = smd2['keywords'].apply(ast.literal_eval)
smd2['cast_size'] = smd2['cast'].apply(lambda x: len(x))
smd2['crew_size'] = smd2['crew'].apply(lambda x: len(x))

In [113]:
smd2.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast_x,crew_x,keywords_x,cast_y,crew_y,keywords_y,cast,crew,keywords,cast_size,crew_size
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",13,106
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",26,16
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",7,4


In the dataset i find that 'cast','crew' and 'keywords' can be considered as metadata.

In [114]:
smd2['crew'][0]

[{'credit_id': '52fe4284c3a36847f8024f49',
  'department': 'Directing',
  'gender': 2,
  'id': 7879,
  'job': 'Director',
  'name': 'John Lasseter',
  'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f4f',
  'department': 'Writing',
  'gender': 2,
  'id': 12891,
  'job': 'Screenplay',
  'name': 'Joss Whedon',
  'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f55',
  'department': 'Writing',
  'gender': 2,
  'id': 7,
  'job': 'Screenplay',
  'name': 'Andrew Stanton',
  'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f5b',
  'department': 'Writing',
  'gender': 2,
  'id': 12892,
  'job': 'Screenplay',
  'name': 'Joel Cohen',
  'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f61',
  'department': 'Writing',
  'gender': 0,
  'id': 12893,
  'job': 'Screenplay',
  'name': 'Alec Sokolow',
  'profile_path': '/v79vlRYi94BZUQnkkyzn

In [115]:
#Function Creation for get the director name from crew columns.
import numpy as np

def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [116]:
smd2['director'] = smd2['crew'].apply(get_director)

So i am getting a new column director which can be treated as metadata further.

In [117]:
smd2['cast'][0]

[{'cast_id': 14,
  'character': 'Woody (voice)',
  'credit_id': '52fe4284c3a36847f8024f95',
  'gender': 2,
  'id': 31,
  'name': 'Tom Hanks',
  'order': 0,
  'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'},
 {'cast_id': 15,
  'character': 'Buzz Lightyear (voice)',
  'credit_id': '52fe4284c3a36847f8024f99',
  'gender': 2,
  'id': 12898,
  'name': 'Tim Allen',
  'order': 1,
  'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'},
 {'cast_id': 16,
  'character': 'Mr. Potato Head (voice)',
  'credit_id': '52fe4284c3a36847f8024f9d',
  'gender': 2,
  'id': 7167,
  'name': 'Don Rickles',
  'order': 2,
  'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'},
 {'cast_id': 17,
  'character': 'Slinky Dog (voice)',
  'credit_id': '52fe4284c3a36847f8024fa1',
  'gender': 2,
  'id': 12899,
  'name': 'Jim Varney',
  'order': 3,
  'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'},
 {'cast_id': 18,
  'character': 'Rex (voice)',
  'credit_id': '52fe4284c3a36847f8024fa5',
  'gender': 2,
  'id': 12900,
 

In [118]:
smd2['cast'] = smd2['cast'].apply(lambda x: [i['name'] for i in x] )
smd2['cast'] = smd2['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [119]:
smd2['cast'][0] #I consider top three actor.

['Tom Hanks', 'Tim Allen', 'Don Rickles']

In [120]:
smd2['keywords']

0        [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
1        [{'id': 10090, 'name': 'board game'}, {'id': 1...
2        [{'id': 1495, 'name': 'fishing'}, {'id': 12392...
3        [{'id': 818, 'name': 'based on novel'}, {'id':...
4        [{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...
                               ...                        
52592    [{'id': 6054, 'name': 'friendship'}, {'id': 20...
52812                 [{'id': 10540, 'name': 'bollywood'}]
52865                 [{'id': 10540, 'name': 'bollywood'}]
53031    [{'id': 1299, 'name': 'monster'}, {'id': 7671,...
53309    [{'id': 6027, 'name': 'music'}, {'id': 10073, ...
Name: keywords, Length: 11367, dtype: object

In [121]:
smd2['keywords'] = smd2['keywords'].apply(lambda x: [i['name'] for i in x] )

In [122]:
smd2['keywords']

0        [jealousy, toy, boy, friendship, friends, riva...
1        [board game, disappearance, based on children'...
2        [fishing, best friend, duringcreditsstinger, o...
3        [based on novel, interracial relationship, sin...
4        [baby, midlife crisis, confidence, aging, daug...
                               ...                        
52592                           [friendship, brick making]
52812                                          [bollywood]
52865                                          [bollywood]
53031    [monster, godzilla, giant monster, destruction...
53309                                 [music, documentary]
Name: keywords, Length: 11367, dtype: object

In [123]:
smd2['cast'] = smd2['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [124]:
smd2['director']

0             John Lasseter
1              Joe Johnston
2             Howard Deutch
3           Forest Whitaker
4             Charles Shyer
                ...        
52592        Gregg Champion
52812     Tinu Suresh Desai
52865    Ashutosh Gowariker
53031          Hideaki Anno
53309            Ron Howard
Name: director, Length: 11367, dtype: object

In [125]:
smd2['director'] = smd2['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd2['director'] = smd2['director'].apply(lambda x: [x,x, x])# to add more priority to director

Here I copy the director nae three times as I consider top three actor for giving equility to the director column and cast column.My tendenct is to keep equal number of person in two columns for each row. 

In [126]:
smd2['director']

0               [johnlasseter, johnlasseter, johnlasseter]
1                  [joejohnston, joejohnston, joejohnston]
2               [howarddeutch, howarddeutch, howarddeutch]
3         [forestwhitaker, forestwhitaker, forestwhitaker]
4               [charlesshyer, charlesshyer, charlesshyer]
                               ...                        
52592        [greggchampion, greggchampion, greggchampion]
52812    [tinusureshdesai, tinusureshdesai, tinusureshd...
52865    [ashutoshgowariker, ashutoshgowariker, ashutos...
53031              [hideakianno, hideakianno, hideakianno]
53309                    [ronhoward, ronhoward, ronhoward]
Name: director, Length: 11367, dtype: object

In [127]:
smd2

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast_x,crew_x,keywords_x,cast_y,crew_y,keywords_y,cast,crew,keywords,cast_size,crew_size,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",13,106,"[johnlasseter, johnlasseter, johnlasseter]"
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",26,16,"[joejohnston, joejohnston, joejohnston]"
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, best friend, duringcreditsstinger, o...",7,4,"[howarddeutch, howarddeutch, howarddeutch]"
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Sava

In [128]:
#Putting index to all the keywords present in the 'keywords' column for each movie.
s = smd2.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)

In [129]:
s

0           jealousy
0                toy
0                boy
0         friendship
0            friends
            ...     
53031    destruction
53031          kaiju
53031          toyko
53309          music
53309    documentary
Length: 78561, dtype: object

In [130]:
s = s.value_counts() #seeing the numbers of available unique keywords.

In [131]:
s = s[s > 1] #considering the keyword which appears more than once.

In [145]:
#Function Creation for considering the more than one appeared keyword in the keywords column of dataset 
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [134]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')   #Finding the root word

In [135]:
#All the operations applying on data keywords column
smd2['keywords'] = smd2['keywords'].apply(filter_keywords)
smd2['keywords'] = smd2['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd2['keywords'] = smd2['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [144]:
smd2['keywords']

0        [jealousi, toy, boy, friendship, friend, rival...
1        [boardgam, disappear, basedonchildren'sbook, n...
2                   [fish, bestfriend, duringcreditssting]
3        [basedonnovel, interracialrelationship, single...
4        [babi, midlifecrisi, confid, age, daughter, mo...
                               ...                        
11362                                         [friendship]
11363                                          [bollywood]
11364                                          [bollywood]
11365     [monster, godzilla, giantmonst, destruct, kaiju]
11366                                 [music, documentari]
Name: keywords, Length: 11367, dtype: object

In [136]:
#assigns a new column to the dataframe which combines all the corresponding unique metadata.

smd2=smd2.assign(soup = smd2.keywords.astype(str) +smd2.crew.astype(str) + smd2.cast.astype(str) +smd2.genres.astype(str))


In [143]:
smd2['soup']

0        ['jealousi', 'toy', 'boy', 'friendship', 'frie...
1        ['boardgam', 'disappear', "basedonchildren'sbo...
2        ['fish', 'bestfriend', 'duringcreditssting'][{...
3        ['basedonnovel', 'interracialrelationship', 's...
4        ['babi', 'midlifecrisi', 'confid', 'age', 'dau...
                               ...                        
11362    ['friendship'][{'credit_id': '544475aac3a36819...
11363    ['bollywood'][{'credit_id': '5951baf692514129c...
11364    ['bollywood'][{'credit_id': '57cd5d3592514179d...
11365    ['monster', 'godzilla', 'giantmonst', 'destruc...
11366    ['music', 'documentari'][{'credit_id': '57057c...
Name: soup, Length: 11367, dtype: object

In [137]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(analyzer='word',min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd2['soup'])

In [138]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [139]:
smd2 = smd2.reset_index()
titles = smd2['title']
indices = pd.Series(smd2.index, index=smd2['title'])

In [140]:
#Calling the get_recommendation function for testing purpose
get_recommendations('The Dark Knight').head(10)

7538                Batman Begins
10909                     Ant-Man
10230            Django Unchained
6160            Kill Bill: Vol. 2
10973                   Insurgent
9831        The Dark Knight Rises
22                      Assassins
266                  Pulp Fiction
1218                     Die Hard
525      The Silence of the Lambs
Name: title, dtype: object

Introducing the Imdb rating formula with the metadata recommendation,Imdb formula is given with the dataset.

In [141]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd2.iloc[movie_indices][['title', 'vote_count', 'vote_average']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.6)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    def weighted_rating(x, m=m, C=C):
      v = x['vote_count']
      R = x['vote_average']
    # Compute the weighted score
      return (v/(v+m) * R) + (m/(m+v) * C)
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [142]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,wr
266,Pulp Fiction,8670,8,7.456254
525,The Silence of the Lambs,4549,8,7.207616
8063,The Prestige,4510,8,7.204172
10230,Django Unchained,10297,7,6.818572
9831,The Dark Knight Rises,9263,7,6.804904
7538,Batman Begins,7511,7,6.776359
5701,Pirates of the Caribbean: The Curse of the Bla...,7191,7,6.770218
4440,Harry Potter and the Philosopher's Stone,7188,7,6.770159
10909,Ant-Man,6029,7,6.744770
11064,Batman v Superman: Dawn of Justice,7189,5,5.536249
